In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

# Read the data from the text file
file_path = '/home/lr/Documents/FUTURUES_PROJECT/DATA/ES_03_22.Last.txt'

In [2]:
try:
    with open(file_path, 'r') as file:
        data_lines = file.readlines()
except FileNotFoundError:
    print(f"Error: File not found at {file_path}")
    raise
except PermissionError:
    print(f"Error: Permission denied when accessing {file_path}")
    raise
except Exception as e:
    print(f"Unexpected error when opening file: {str(e)}")
    raise

print(f"Successfully loaded file with {len(data_lines)} lines")

Successfully loaded file with 92207 lines


In [3]:
# Initialize lists to store data
timestamps = []
open_prices = []
high_prices = []
low_prices = []
close_prices = []
volumes = []

# Parse each line
for line in data_lines:
    # Split by semicolon
    parts = line.strip().split(';')
    if len(parts) != 6:
        print(f"Skipping malformed line: {line}")
        continue
    
    # Parse timestamp (format: YYYYMMDD HHMMSS)
    date_time_str = parts[0]
    try:
        timestamp = datetime.strptime(date_time_str, '%Y%m%d %H%M%S')
        timestamps.append(timestamp)
    except ValueError:
        print(f"Skipping line with invalid datetime: {line}")
        continue
    
    # Parse OHLC and volume data
    try:
        open_prices.append(float(parts[1]))
        high_prices.append(float(parts[2]))
        low_prices.append(float(parts[3]))
        close_prices.append(float(parts[4]))
        volumes.append(int(parts[5]))
    except ValueError:
        print(f"Skipping line with invalid numeric data: {line}")
        continue

In [4]:
# Create DataFrame
df = pd.DataFrame({
    'timestamp': timestamps,
    'open': open_prices,
    'high': high_prices,
    'low': low_prices,
    'close': close_prices,
    'volume': volumes
})

In [5]:
# Set timestamp as index
df.set_index('timestamp', inplace=True)

# Add trading session information
df['session'] = 'ETH'  # Default to Extended Trading Hours
# Regular Trading Hours (RTH) is typically 9:30 AM - 4:00 PM ET
df.loc[df.index.hour.isin(range(9, 16)) & 
       ((df.index.hour != 9) | (df.index.minute >= 30)), 'session'] = 'RTH'

# Check for data quality issues
print(f"Original data shape: {df.shape}")

Original data shape: (92207, 6)


In [6]:
# Check for duplicated timestamps
duplicates = df.index.duplicated()
if duplicates.any():
    print(f"Found {duplicates.sum()} duplicate timestamps")
    # Either keep first occurrence or handle as needed
    df = df[~duplicates]

# Check for missing data
print(f"Missing values per column:\n{df.isna().sum()}")

# Check for zero or negative prices
if (df[['open', 'high', 'low', 'close']] <= 0).any().any():
    print("Warning: Found zero or negative prices")

# Check for high-low inconsistency
inconsistent = (df['high'] < df['low']).any()
if inconsistent:
    print("Warning: Found high < low inconsistencies")

# Check for OHLC inconsistencies
ohlc_issues = ((df['open'] > df['high']) | 
               (df['open'] < df['low']) | 
               (df['close'] > df['high']) | 
               (df['close'] < df['low']))
if ohlc_issues.any():
    print(f"Found {ohlc_issues.sum()} OHLC relationship inconsistencies")

Missing values per column:
open       0
high       0
low        0
close      0
volume     0
session    0
dtype: int64


In [7]:
# Calculate basic statistics
print("\nData Statistics:")
print(f"Date range: {df.index.min()} to {df.index.max()}")
print(f"Number of data points: {len(df)}")
print(f"RTH sessions: {(df['session'] == 'RTH').sum()}")
print(f"ETH sessions: {(df['session'] == 'ETH').sum()}")
print("\nPrice statistics:")
print(df[['open', 'high', 'low', 'close']].describe())
print("\nVolume statistics:")
print(df['volume'].describe())


Data Statistics:
Date range: 2021-12-05 16:31:00 to 2022-03-10 05:00:00
Number of data points: 92207
RTH sessions: 26125
ETH sessions: 66082

Price statistics:
               open          high           low         close
count  92207.000000  92207.000000  92207.000000  92207.000000
mean    4529.988724   4531.034306   4528.936127   4529.983279
std      171.720000    171.298221    172.135816    171.717974
min     4104.250000   4105.500000   4101.750000   4104.250000
25%     4380.000000   4381.250000   4378.500000   4380.000000
50%     4547.500000   4548.500000   4546.500000   4547.500000
75%     4678.000000   4678.750000   4677.250000   4678.000000
max     4807.000000   4808.250000   4806.500000   4807.250000

Volume statistics:
count     92207.000000
mean       1171.181559
std        2500.377578
min           1.000000
25%         103.000000
50%         298.000000
75%        1293.000000
max      120700.000000
Name: volume, dtype: float64


In [13]:
# Add derived columns useful for analysis
df['bar_range'] = df['high'] - df['low']
df['bar_return'] = df['close'].pct_change()

# Display first few rows of cleaned data
print("\nCleaned data sample:")
display(df.head())

# Save cleaned data
df.to_csv('/home/lr/Documents/FUTURUES_PROJECT/Data/cleaned_futures_data.csv')
print("Cleaned data saved to 'cleaned_futures_data.csv'")


Cleaned data sample:


,open,high,low,close,volume,session,bar_range,bar_return
timestamp,,,,,,,,
2021-12-05 16:31:00,4527.0,4527.00,4527.00,4527.00,12,ETH,0.00,NaN
2021-12-05 23:01:00,4536.0,4542.00,4531.25,4531.25,72,ETH,10.75,0.000939
2021-12-05 23:02:00,4531.0,4534.25,4529.25,4532.50,12,ETH,5.00,0.000276
2021-12-05 23:03:00,4532.5,4536.00,4532.00,4536.00,8,ETH,4.00,0.000772
2021-12-05 23:04:00,4538.5,4540.00,4537.00,4540.00,5,ETH,3.00,0.000882


OSError: Cannot save file into a non-existent directory: '/home/lr/Documents/FUTURUES_PROJECT/Data'